**Run CR-CC(2,3) using PySCF**

In [15]:
from pyscf import gto, scf, cc
import numpy as np

In [30]:
### Running a basic CC calculation

mol = gto.Mole()
mol.atom = '''
    H  0  0  0
    F  0  0  1.1'''
mol.basis = 'ccpvdz'
mol.symmetry=True
mol.spin = 0
mol.verbose=1
mol.charge=0
mol.build()

## Run Restricted Hartree-Fock
mf = scf.RHF(mol)
mf.kernel()
print("RHF Energy:    ",mf.e_tot,"hartree")
## Run CCSD
mycc = cc.CCSD(mf)
mycc.verbose=1
mycc.kernel()
print("CCSD Energy:   ",mycc.e_tot,"hartree")
ccsdpt = mycc.ccsd_t()
print("CCSD(T) Energy:",mycc.e_tot + ccsdpt,"hartree")




RHF Energy:     -99.98739744034886 hartree
CCSD Energy:    -100.20373507082256 hartree
CCSD(T) Energy: -100.20614876218058 hartree


From PySCf docs:
'''restricted CCSD

    Attributes:
        verbose : int
            Print level.  Default value equals to :class:`Mole.verbose`
        max_memory : float or int
            Allowed memory in MB.  Default value equals to :class:`Mole.max_memory`
        conv_tol : float
            converge threshold.  Default is 1e-7.
        conv_tol_normt : float
            converge threshold for norm(t1,t2).  Default is 1e-5.
        max_cycle : int
            max number of iterations.  Default is 50.
        diis_space : int
            DIIS space size.  Default is 6.
        diis_start_cycle : int
            The step to start DIIS.  Default is 0.
        iterative_damping : float
            The self consistent damping parameter.
        direct : bool
            AO-direct CCSD. Default is False.
        async_io : bool
            Allow for asynchronous function execution. Default is True.
        incore_complete : bool
            Avoid all I/O (also for DIIS). Default is False.
        level_shift : float
            A shift on virtual orbital energies to stablize the CCSD iteration
        frozen : int or list
            If integer is given, the inner-most orbitals are frozen from CC
            amplitudes.  Given the orbital indices (0-based) in a list, both
            occupied and virtual orbitals can be frozen in CC calculation.

            >>> mol = gto.M(atom = 'H 0 0 0; F 0 0 1.1', basis = 'ccpvdz')
            >>> mf = scf.RHF(mol).run()
            >>> # freeze 2 core orbitals
            >>> mycc = cc.CCSD(mf).set(frozen = 2).run()
            >>> # freeze 2 core orbitals and 3 high lying unoccupied orbitals
            >>> mycc.set(frozen = [0,1,16,17,18]).run()

    Saved results

        converged : bool
            CCSD converged or not
        e_corr : float
            CCSD correlation correction
        e_tot : float
            Total CCSD energy (HF + correlation)
        t1, t2 :
            T amplitudes t1[i,a], t2[i,j,a,b]  (i,j in occ, a,b in virt)
        l1, l2 :
            Lambda amplitudes l1[i,a], l2[i,j,a,b]  (i,j in occ, a,b in virt)

In [52]:
### Now after running CCSD we already have T1 and T2 amplitudes
### To get them out of PySCf we do the following

T1 = mycc.t1
T2 = mycc.t2

In [53]:
### Next we want the lambda vectors, but for that we need to
### solve the left-CCSD equations. We can do that in the
### following way

L1, L2 = mycc.solve_lambda()